## Set the repository path

In [ ]:
pyCPT_path = '../'

## Import modules

In [ ]:
# Add the repository path into python path
import sys
sys.path.append(pyCPT_path)

# Import modules
import pyCPT
import post_process
import numpy as np

## Read the NGES dataset
#### Note: 1) Data file should contain 3 columns: ( depth, Fr, Qt ); 2) Data should start from the 1st row!

In [ ]:
# By default, model selection using BIC is performed
datapath = '../test_data/NGES_data.csv'
cpt1 = pyCPT.CPT(datapath)

In [ ]:
# If explicitly turn off model selection option, no optimal number of cluster will be inferred.
datapath = '../test_data/NGES_data.csv'
cpt2 = pyCPT.CPT(datapath,do_model_selection=False)

## Extract soil segments

In [ ]:
# if no prior information is available
cpt1.segmentation(num_of_iter=200, start_iter=100)

In [ ]:
# if prior information is available
# prior knowledge of the center of each cluster
prior_mu = np.array([[0.3, 1.8],
       [0, 1],
       [0.6, 1.],
       [0.5, 1.6],
       [0.4, 2.4],
       [0.8, 1.7],
       [0.4, 1.2]])
# the standard deviation of the center coordinates
prior_mu_cov = [1,0.0001,2,1,2,1,2]
# prior knowledge of the covariance matrix of each cluster
prior_cov = np.array([[[ 0.004,  0.002],
        [ 0.002,  0.01]],

       [[0.005, 0],
        [0,  0.005]],

       [[ 0.002,  0.002],
        [ 0.002,  0.02]],

       [[ 0.006, -0.004],
        [-0.004,  0.006]],

       [[ 0.04, -0.04],
        [-0.04,  0.05]],

       [[ 0.002,  0.0002],
        [ 0.0002,  0.03 ]],

       [[ 0.01, -0.02],
        [-0.02,  0.05]]])

In [ ]:
cpt2.segmentation(num_of_iter=200, start_iter=100, prior_mu=prior_mu, prior_mu_cov=prior_mu_cov, prior_cov=prior_cov)

## Soil interpretation

In [ ]:
cpt1.detect_layers()
cpt2.detect_layers()

## Visualization

In [ ]:
post_process.plot_feature(cpt1.element,xlabel=['Fr','Qt'], scatter=False) # show the CPT data
post_process.plot_image(cpt1.element, cpt1.element.label_map_est) # show the spatial pattern
pyCPT.mixture_plot_robertson_chart(cpt1, aspect=0.8) # show the statistical pattern
pyCPT.plot_layers(cpt1) # show the soil layers

In [ ]:
post_process.plot_feature(cpt2.element,xlabel=['Fr','Qt'], scatter=False) # show the CPT data
post_process.plot_image(cpt2.element, cpt2.element.label_map_est) # show the spatial pattern
pyCPT.mixture_plot_robertson_chart(cpt2, aspect=0.8) # show the statistical pattern
pyCPT.plot_layers(cpt2) # show the soil layers

#### Identified issue: When you test the repository by using the on-line interactive binder version, some noise contour lines could show up in the contour plot on the Roberson chart. This is an identified issue and probabily related to the version of the package "matplotlib" in MyBinder.org. The colone version from the github repository does not have such behavior.

In [ ]:
pyCPT.write_mat('test_output_1.mat',cpt1)
pyCPT.write_mat('test_output_2.mat',cpt2)